REDDIT TEXT ANALYSIS

Retrieves 5000 submissions from the 'stocks' subreddit, analyzing word frequency for the top 10 commenters (based on score) and overall frequencies to compute PMI values for each word. For each of the top commenters, their top 10 PMI rated words are listed. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import praw
from cytoolz import take
from cytoolz import concat

In [2]:
reddit = praw.Reddit(client_id='2YQV8no_mDXhEg', client_secret='WMHfDZ6YxuhmUNvzg2jrrxYBS8c',
                    user_agent='python program')

In [3]:
subreddit = reddit.subreddit('stocks')
s = next(subreddit.hot())
s

Submission(id='7cnoeb')

In [4]:
vars(s)
#permalink to access info from outside
#selftext is the desc of text
#what things are conatined in this s object, like author, comment, banned, etc...

{'_comments_by_id': {},
 '_fetched': False,
 '_flair': None,
 '_info_params': {},
 '_mod': None,
 '_reddit': <praw.reddit.Reddit at 0x117024048>,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': Redditor(name='AutoModerator'),
 'author_flair_css_class': None,
 'author_flair_text': None,
 'banned_at_utc': None,
 'banned_by': None,
 'brand_safe': True,
 'can_gild': False,
 'can_mod_post': False,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'best',
 'contest_mode': False,
 'created': 1510614668.0,
 'created_utc': 1510585868.0,
 'distinguished': 'moderator',
 'domain': 'self.stocks',
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'hidden': False,
 'hide_score': False,
 'id': '7cnoeb',
 'is_crosspostable': False,
 'is_reddit_media_domain': False,
 'is_self': True,
 'is_video': False,
 'likes': None,
 'link_flair_css_class': None,
 'link_flair_text': None,
 'locked': False,
 'media': None,
 'media_embed': {},
 'mod_note': None,
 'mod_reason_by': No

In [5]:
s.comments.replace_more(limit=None)
comments = [{'author':c.author.name, 'body':c.body, 'score':c.score}
            for c in s.comments.list() if c.author]
data = pd.DataFrame(comments)
data

,author,body,score
0,provoko,I just added an automod rule that redirects us...,1
1,MartyMoho,125% MU,68
2,Nicksmells34,18% in ACTVI \n8.9% NTDOY\n15% in PULM\n33% in...,21
3,KillerMagikarp,MGK - 5.46%\nVFH - 9.87%\nAAPL - 34.56%\nXPO -...,21
4,plmattice,Investment | Percentage | Type\n:--------|:---...,18
5,cheeeeeese,25% technology - hardware\n\n20% oil & gas\n\n...,14
6,Clan57,"HMMJ, WEED.TO, SHOP.TO, INTC, TCEHY, soon BOTZ...",13
7,dundrmiflinthisispam,42% MU\n\n26% FB\n\n17% AMD\n\n15% AAPL\n\n.01...,10
8,FIinvestor,25M - 100k\n\nI have been moving more towards ...,6
9,ne0ven0m,"After a month of chasing meme stocks, I've fin...",7


In [6]:
data.to_csv('stocks-mini.csv')

In [7]:
from ipywidgets import IntProgress
from IPython.display import display

In [8]:
%%time

bar = IntProgress(min=0, max=5000)
display(bar)

comments = [ ]
for s in take(5000, subreddit.submissions()):
    bar.value += 1
    s.comments.replace_more(limit=None)
    comments.extend([{'author':c.author.name, 
                      'body':c.body, 
                      'score':c.score} 
                    for c in s.comments.list() if c.author])

Widget Javascript not detected.  It may not be installed or enabled properly.


CPU times: user 39.3 s, sys: 2.92 s, total: 42.2 s
Wall time: 20min 51s


In [9]:
data = pd.DataFrame(comments)
data.to_csv('stocks.csv', index=False)

In [10]:
len(data)

63576

In [11]:
#data = pd.read_csv('stocks.csv')

#CREATE BAG OF WORDS
data['bow'] = data['body'].str.lower().str.replace(r'(\W|\d)+', ' ').str.split()
data.to_csv('stocks.csv', index=False)
data['bow']

0        [ok, you, lost, me, at, deep, state, it, s, no...
1        [please, get, your, information, from, a, repu...
2        [lol, has, nothing, to, do, with, a, deep, sta...
3        [professional, money, always, sells, into, str...
4                     [https, finviz, com, screener, ashx]
5        [i, think, i, m, misunderstanding, you, but, i...
6        [it, doesn, t, make, any, sense, to, filter, b...
7        [lol, no, nobody, else, has, ever, though, of,...
8        [did, you, just, tell, op, to, go, fuck, him, ...
9        [i, don, t, think, so, lol, i, really, don, t,...
10       [i, assume, that, would, involve, going, long,...
11       [pg, has, been, beaten, up, recently, yield, i...
12       [hah, yes, just, started, looking, into, it, t...
13                         [nothing, can, go, wrong, here]
14       [depends, if, you, want, to, make, money, or, ...
15                                         [r, weedstocks]
16                    [canopy, growth, corporation, twmj

In [12]:
data

,author,body,score,bow
0,ink-ling,"OK, you lost me at deep state. It's not any ne...",8,"[ok, you, lost, me, at, deep, state, it, s, no..."
1,naitdawggg,PLEASE get your information from a reputable s...,4,"[please, get, your, information, from, a, repu..."
2,BaggySphere,Lol has nothing to do with a deep state.\n\n-F...,4,"[lol, has, nothing, to, do, with, a, deep, sta..."
3,BannerlordAdmirer,Professional money always sells into strength....,1,"[professional, money, always, sells, into, str..."
4,PM_ME_BOOBPIX,https://finviz.com/screener.ashx,2,"[https, finviz, com, screener, ashx]"
5,lioninn22,"I think I'm misunderstanding you, but I would...",1,"[i, think, i, m, misunderstanding, you, but, i..."
6,tocommaclub,It doesn't make any sense to filter by stock p...,1,"[it, doesn, t, make, any, sense, to, filter, b..."
7,We_Killed_Satoshi,LOL no. Nobody else has ever though of using a...,0,"[lol, no, nobody, else, has, ever, though, of,..."
8,whitethunder9,Did you just tell OP to go fuck him/herself?,1,"[did, you, just, tell, op, to, go, fuck, him, ..."
9,lioninn22,"I don't think so lol, I really don't understan...",1,"[i, don, t, think, so, lol, i, really, don, t,..."


In [13]:
#FIND FREQUENCIES FOR ALL WORDS IN SAMPLE
freq = pd.DataFrame({'all': pd.value_counts(list(concat(data['bow'])))})
freq

,all
the,77045
i,56943
a,55074
to,54700
and,45496
it,39955
you,36732
is,34922
of,33632
in,32562


In [15]:
#GET TOP 10 COMMENTERS BY USING SCORE
top_scores = data.groupby('author')['score'].mean().nlargest(10).reset_index(name='score')
top_authors = top_scores['author']

#CALCULATE PMI FOR TOP 10 COMMENTERS
for d in data:
    for a in top_authors:
        freq[a] = pd.value_counts(list(concat(data[data['author'] == a]['bow'])))
        freq[a+'_pmi'] = np.log2( (freq[a] * np.sum(freq['all'])) / 
                        (freq['all'] * np.sum(freq[a])) )
        
freq.fillna(0, inplace=True)
freq

,all,PitchforkAssistant,PitchforkAssistant_pmi,Iantompkins10,Iantompkins10_pmi,MentalRental,MentalRental_pmi,BrrrtgoesGermany,BrrrtgoesGermany_pmi,ryan4664,...,foxymanchester,foxymanchester_pmi,davidthearmo,davidthearmo_pmi,Juventus19,Juventus19_pmi,lokhagos,lokhagos_pmi,verygoodguy10,verygoodguy10_pmi
the,77045,1.0,0.755101,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,1.0,1.520636,0.0,0.000000,0.0,0.000000,27.0,0.971743,0.0,0.000000
i,56943,0.0,0.000000,0.0,0.0,1.0,0.278747,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,1.0,0.819315,5.0,-1.025034,0.0,0.000000
a,55074,0.0,0.000000,0.0,0.0,1.0,0.326894,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,23.0,1.224747,1.0,2.004966
to,54700,0.0,0.000000,0.0,0.0,2.0,1.336725,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,2.0,1.877293,11.0,0.170447,0.0,0.000000
and,45496,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,6.0,-0.438221,0.0,0.000000
it,39955,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,1.0,2.467961,0.0,0.000000,1.0,1.330458,1.0,-2.835819,0.0,0.000000
you,36732,1.0,1.823765,0.0,0.0,2.0,1.911228,1.0,2.210788,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,22.0,1.744951,0.0,0.000000
is,34922,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,1.0,1.524698,7.0,0.165775,0.0,0.000000
of,33632,0.0,0.000000,0.0,0.0,1.0,1.038431,0.0,0.000000,1.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,11.0,0.872154,0.0,0.000000
in,32562,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,9.0,0.629292,0.0,0.000000


In [16]:
#list for dictionaries
top_commenters = []
#DataFrame for finding top 10 pmi words based on author
top_pmi = pd.DataFrame()
for d in data:
    for a in top_authors:
        top_pmi[a] = [x for x in dict(freq[a+'_pmi'][freq['all']>25].sort_values(ascending=False)[:10])]
        top_commenters.append({a: top_pmi[a].tolist()})
        #dictionary added to list, key is author and values are list of top 10 pmi words for author

top_pmi

,PitchforkAssistant,Iantompkins10,MentalRental,BrrrtgoesGermany,ryan4664,foxymanchester,davidthearmo,Juventus19,lokhagos,verygoodguy10
0,monitor,steve,owe,trillion,cold,prior,patient,friend,deposit,published
1,flip,ui,chunk,talking,bitch,called,being,till,damage,fake
2,upside,idk,joke,worth,ex,every,not,adding,dream,date
3,too,reverse,margin,ago,an,who,ui,retirement,home,order
4,down,millions,broker,over,my,same,ignore,certain,mall,was
5,go,funny,holdings,about,of,year,wont,best,coverage,no
6,also,wont,large,years,unfortunately,people,shitty,portfolio,homes,buy
7,do,shitty,means,was,idk,for,spent,pretty,car,this
8,up,spent,trading,what,millions,it,idk,keep,insurance,a
9,your,n,even,are,funny,the,n,got,fault,ignore


In [17]:
#PRINT LIST OF DICTIONARIES WITH LIST VALUES
for t in top_commenters:       
    for key, values in t.items():
        print(key,' : ',values)
        print("\n")

PitchforkAssistant  :  ['monitor', 'flip', 'upside', 'too', 'down', 'go', 'also', 'do', 'up', 'your']


Iantompkins10  :  ['steve', 'ui', 'idk', 'reverse', 'millions', 'funny', 'wont', 'shitty', 'spent', 'n']


MentalRental  :  ['owe', 'chunk', 'joke', 'margin', 'broker', 'holdings', 'large', 'means', 'trading', 'even']


BrrrtgoesGermany  :  ['trillion', 'talking', 'worth', 'ago', 'over', 'about', 'years', 'was', 'what', 'are']


ryan4664  :  ['cold', 'bitch', 'ex', 'an', 'my', 'of', 'unfortunately', 'idk', 'millions', 'funny']


foxymanchester  :  ['prior', 'called', 'every', 'who', 'same', 'year', 'people', 'for', 'it', 'the']


davidthearmo  :  ['patient', 'being', 'not', 'ui', 'ignore', 'wont', 'shitty', 'spent', 'idk', 'n']


Juventus19  :  ['friend', 'till', 'adding', 'retirement', 'certain', 'best', 'portfolio', 'pretty', 'keep', 'got']


lokhagos  :  ['deposit', 'damage', 'dream', 'home', 'mall', 'coverage', 'homes', 'car', 'insurance', 'fault']


verygoodguy10  :  ['published